<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">Sentiment Analysis of Reddit Data using Reddit API</h1>

In this live coding session, we leverage the Python Reddit API Wrapper (`PRAW`) to retrieve data from subreddits on [Reddit](https://www.reddit.com), and perform sentiment analysis using [`pipelines`](https://huggingface.co/docs/transformers/main_classes/pipelines) from [HuggingFace ( 🤗 the GitHub of Machine Learning )](https://techcrunch.com/2022/05/09/hugging-face-reaches-2-billion-valuation-to-build-the-github-of-machine-learning/), powered by [transformer](https://arxiv.org/pdf/1706.03762.pdf).

## Objectives

At the end of the session, you will 

- know how to work with APIs
- feel more comfortable navigating thru documentation, even inspecting the source code
- understand what a `pipeline` object is in HuggingFace
- perform sentiment analysis using `pipeline`
- run a python script in command line and get the results

## How to Submit

- At the end of each task, commit* the work into the repository you created before the assignment
- After completing all three tasks, make sure to push the notebook containing all code blocks and output cells to your repository you created before the assignment
- Submit the link to the notebook in Canvas

\***NEVER** commit a notebook displaying errors unless it is instructed otherwise. However, commit often; recall git ABC = **A**lways **B**e **C**ommitting.

## Tasks

### Task I: Instantiate a Reddit API Object

The first task is to instantiate a Reddit API object using [PRAW](https://praw.readthedocs.io/en/stable/), through which you will retrieve data. PRAW is a wrapper for [Reddit API](https://www.reddit.com/dev/api) that makes interacting with the Reddit API easier unless you are already an expert of [`requests`](https://docs.python-requests.org/en/latest/).

#### 1. Install packages

Please ensure you've ran all the cells in the `imports.ipynb`, located [here](https://github.com/FourthBrain/MLE-8/blob/main/assignments/week-3-analyze-sentiment-subreddit/imports.ipynb), to make sure you have all the required packages for today's assignment.

####  2. Create a new app on Reddit 

Create a new app on Reddit and save secret tokens; refer to [post in medium](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c) for more details.

- Create a Reddit account if you don't have one, log into your account.
- To access the API, we need create an app. Slight updates, on the website, you need to navigate to `preference` > `app`, or click [this link](https://www.reddit.com/prefs/apps) and scroll all the way down. 
- Click to create a new app, fill in the **name**, choose `script`, fill in  **description** and **redirect uri** ( The redirect URI is where the user is sent after they've granted OAuth access to your application (more info [here](https://github.com/reddit-archive/reddit/wiki/OAuth2)) For our purpose, you can enter some random url, e.g., www.google.com; as shown below.


    <img src="https://miro.medium.com/max/700/1*lRBvxpIe8J2nZYJ6ucMgHA.png" width="500"/>
- Jot down `client_id` (left upper corner) and `client_secret` 

    NOTE: CLIENT_ID refers to 'personal use script" and CLIENT_SECRET to secret.
    
    <div>
    <img src="https://miro.medium.com/max/700/1*7cGAKth1PMrEf2sHcQWPoA.png" width="300"/>
    </div>

- Create `secrets_reddit.py` in the same directory with this notebook, fill in `client_id` and `secret_id` obtained from the last step. We will need to import those constants in the next step.
    ```
    REDDIT_API_CLIENT_ID = "client_id"
    REDDIT_API_CLIENT_SECRET = "secret_id"
    REDDIT_API_USER_AGENT = "any string except bot; ex. My User Agent"
    ```
- Add `secrets_reddit.py` to your `.gitignore` file if not already done. NEVER push credentials to a repo, private or public. 

#### 3. Instantiate a `Reddit` object

Now you are ready to create a read-only `Reddit` instance. Refer to [documentation](https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html) when necessary.

In [1]:
import praw
import secrets_reddit

# Create a Reddit object which allows us to interact with the Reddit API
reddit = praw.Reddit(
    client_id=secrets_reddit.REDDIT_API_CLIENT_ID,
    client_secret=secrets_reddit.REDDIT_API_CLIENT_SECRET,
    user_agent=secrets_reddit.REDDIT_API_USER_AGENT
)

In [2]:
print(reddit) 

<details>
<summary>Expected output:</summary>   

```<praw.reddit.Reddit object at 0x10f8a0ac0>```
</details>

#### 4. Instantiate a `subreddit` object

Lastly, create a `subreddit` object for your favorite subreddit and inspect the object. The expected outputs you will see are from `r/machinelearning` unless otherwise specified.

In [3]:
subreddit = reddit.subreddit("machinelearning")

What is the display name of the subreddit? machinelearning

In [4]:
subreddit.display_name

'machinelearning'

<details>
<summary>Expected output:</summary>   

    machinelearning
</details>

How about its title, is it different from the display name? 
Machine Learning
The title is properly formatted with capital letters and a space.

In [5]:
subreddit.title

'Machine Learning'

<details>
<summary>Expected output:</summary>   

    Machine Learning
</details>

Print out the description of the subreddit:

In [6]:
print(subreddit.description[:400])

**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q


<details>
<summary>Expected output:</summary>

    **[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
    --------
    +[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
    --------
    +[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
    --------
    +[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
    --------
    +[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict
</details>

### Task II: Parse comments

#### 1. Top Posts of All Time

Find titles of top 10 posts of **all time** from your favorite subreddit. Refer to [Obtain Submission Instances from a Subreddit Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html)) if necessary. Verify if the titles match what you read on Reddit.

In [7]:
# try run this line, what do you see? press q once you are done
?subreddit.top 

In [8]:
for submission in subreddit.top(time_filter="all", limit=10):
    print(submission.title)

[Project] From books to presentations in 10s with AR + ML
[D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
[R] First Order Motion Model applied to animate paintings
[N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
[D] This AI reveals how much time politicians stare at their phone at work
[D] Types of Machine Learning Papers
[D] The machine learning community has a toxicity problem
I made a robot that punishes me if it detects that if I am procrastinating on my assignments [P]
[Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
[P] Using oil portraits and First Order Model to bring the paintings back to life


<details> <summary>Expected output:</summary>

    [Project] From books to presentations in 10s with AR + ML
    [D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
    [R] First Order Motion Model applied to animate paintings
    [N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
    [D] This AI reveals how much time politicians stare at their phone at work
    [D] Types of Machine Learning Papers
    [D] The machine learning community has a toxicity problem
    [Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
    [P] Using oil portraits and First Order Model to bring the paintings back to life
    [D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)    
</details>

#### 2. Top 10 Posts of This Week

What are the titles of the top 10 posts of **this week** from your favorite subreddit?

In [9]:
for submission in subreddit.top(limit=10, time_filter="week"):
    print(submission.title)

[P] Run Stable Diffusion locally with a web UI + artist workflow video
[P] Einstein Instant NeRF
[D] How to Run Stable Diffusion (Locally and in Colab)
[D][N]"Mudge learned that Twitter had never acquired proper legal rights to training material used to build Twitter's key Machine Learning models. The Machine Learning models at issue were some of the core models running the company's most basic products, like which Tweets to show each user."
[D] ML for Good
[D] What are some dead ideas in machine learning or machine learning textbooks?
[D] "A majority of respondents think that the scientific value of the majority of work in NLP is dubious"
[P] Run stable diffusion in google colab including image2image and inpainting
[D] A thought I had on Yann LeCun's recent paper "A Path Towards Autonomous Machine Intelligence"
[D] What does production look like in your case?


<details><summary>Expected output:</summary>

    [N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.
    [R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo
    [P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.
    [R] Meta is releasing a 175B parameter language model
    [N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics
    [P] T-SNE to view and order your Spotify tracks
    [D] : HELP Finding a Book - A book written for Google Engineers about foundational Math to support ML
    [R] Scaled up CLIP-like model (~2B) shows 86% Zero-shot on Imagenet
    [D] Do you use NLTK or Spacy for text preprocessing?
    [D] Democratizing Diffusion Models - LDMs: High-Resolution Image Synthesis with Latent Diffusion Models, a 5-minute paper summary by Casual GAN Papers
</details>

💽❓ Data Question:

Check out what other attributes the `praw.models.Submission` class has in the [docs](https://praw.readthedocs.io/en/stable/code_overview/models/submission.html). 

1. After having a chance to look through the docs, is there any other information that you might want to extract? How might this additional data help you?

Additional information I would want to extract are 'score' (the number of upvotes for the submission), 'upvote_ratio' (the percentage of upvotes from all votes on the submission), and 'edited' (whether or not the submission has been edited)

By knowing amount of upvotes, it will allow me to understand which submissions have approval or support by the users. It will help in distributing the data evenly for the training and test sets. Taking a closer look at edited post is important to see if the edit is changing the sentiment although the message of the post remains the same. The pre-trained model can change it's prediction depending on the authors choose of words.

Write a sample piece of code below extracting three additional pieces of information from the submission below.

In [28]:
for submission in subreddit.top(limit=10, time_filter="week"):
    print("Score", submission.score)
    print("Upvote ratio", submission.upvote_ratio)
    print("Edited", submission.edited)

Score 1190
Upvote ratio 0.99
Edited False
Score 339
Upvote ratio 0.96
Edited False
Score 268
Upvote ratio 0.98
Edited False
Score 227
Upvote ratio 0.92
Edited False
Score 221
Upvote ratio 0.94
Edited False
Score 217
Upvote ratio 0.98
Edited False
Score 172
Upvote ratio 0.96
Edited False
Score 124
Upvote ratio 0.98
Edited 1661460439.0
Score 110
Upvote ratio 0.95
Edited False
Score 107
Upvote ratio 0.93
Edited False


💽❓ Data Question:

2. Is there any information available that might be a concern when it comes to Ethical Data?

The author's name is available can be a concern for the users privacy. Although most usernames are anonymous, there are some that may be using their real names. Unless a user gives consent, the information gathered should remain private to protect the individual. 

#### 3. Comment Code

Add comments to the code block below to describe what each line of the code does (Refer to [Obtain Comment Instances Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html) when necessary). The code is adapted from [this tutorial](https://praw.readthedocs.io/en/stable/tutorials/comments.html)

The purpose is 
1. to understand what the code is doing 
2. start to comment your code whenever it is not self-explantory if you have not (others will thank you, YOU will thank you later 😊) 

In [11]:
%%time
from praw.models import MoreComments

# List of all top comments
top_comments = []

# Iterate top 10 subreddit post
for submission in subreddit.top(limit=10):
    # Iterate comments for post
    for top_level_comment in submission.comments:
        # Verifies that comment is type of MoreComments and skips to next comment
        if isinstance(top_level_comment, MoreComments):
            continue
        # Add comment to list
        top_comments.append(top_level_comment.body)

CPU times: user 330 ms, sys: 24.6 ms, total: 354 ms
Wall time: 15.2 s


#### 4. Inspect Comments

How many comments did you extract from the last step? 741 

Examine a few comments. 

In [12]:
len(top_comments) # the answer may vary 693 for r/machinelearning

741

In [27]:
import random

[random.choice(top_comments) for i in range(5)]

['re-animator',
 "The focus on quantity over quality is a big one. We should be focusing on quality research instead of trying to increase our publication count. Also, the focus on just throwing more data at larger models like GPT-3 is a super bad direction for the field to be going in. Rather than actual innovation it's just larger models and more data and making things even more exclusive to the large companies and labs with 1000s of GPUs and tons of funding and resources.",
 'Bleah.  Saw this in Interview With the Vampire 30 years ago.',
 'Still incredibly cool',
 "Lol at Jan Jambon. Glad I didn't vote for him."]

<details> <summary>Some of the comments from `r/machinelearning` subreddit are:</summary>

    ['Awesome visualisation',
    'Similar to a stack or connected neurons.',
    'Will this Turing pass the Turing Test?']
</details>

💽❓ Data Question:

3. After having a chance to review a few samples of 5 comments from the subreddit, what can you say about the data? 

Data needs to be cleaned. It can be improved by removing comments, links and emojis that are irrelavant and that have links and emojis. By doing this, the sentiment analysis can run more effectively and may provide results that are more accurate.

HINT: Think about the "cleanliness" of the data, the content of the data, think about what you're trying to do - how does this data line up with your goal?

#### 5. Extract Top Level Comment from Subreddit `TSLA`.

Write your code to extract top level comments from the top 10 topics of a time period, e.g., year, from subreddit `TSLA` and store them in a list `top_comments_tsla`.  

In [14]:
tsla_subreddit = reddit.subreddit("tsla")

# List of all top comments
top_comments_tsla = []

# Iterate top 10 subreddit post
for submission in tsla_subreddit.top(limit=10):
    # Iterate top comments for post
    for top_level_comment in submission.comments:
        # Verify that comments is data type MoreComments 
        if isinstance(top_level_comment, MoreComments):
            continue
        # Add comment to list
        top_comments_tsla.append(top_level_comment.body)

In [15]:
len(top_comments_tsla) # Expected: 174 for r/machinelearning

170

In [26]:
[random.choice(top_comments_tsla) for i in range(3)]

['I bot 1 share today u/596',
 'How come we are all out of money?',
 '75 Shares baby \n\n90% of my portfolio']

<details>
<summary>Some of the comments from `r/TSLA` subreddit:</summary>

    ['I bought puts',
    '100%',
    'Yes. And I’m bag holding 1200 calls for Friday and am close to throwing myself out the window']
</details>

💽❓ Data Question:

4. Now that you've had a chance to review another subreddits comments, do you see any differences in the kinds of comments either subreddit has - and how might this relate to bias?

The r/TSLA subreddit many of the comments are about a users involvement with Tesla. This is an example of sample bias as users' may be commenting based on their views on the value of the company as an investment and not about their products.

### Task III: Sentiment Analysis

Let us analyze the sentiment of comments scraped from `r/TSLA` using a pre-trained HuggingFace model to make the inference. Take a [Quick tour](https://huggingface.co/docs/transformers/quicktour). 

#### 1. Import `pipeline`

In [17]:
from transformers import pipeline

#### 2. Create a Pipeline to Perform Task "sentiment-analysis"

In [18]:
sentiment_model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


#### 3. Get one comment from list `top_comments_tsla` from Task II - 5.

In [19]:
comment = random.choice(top_comments_tsla)

In [20]:
comment

'TSLA $2,500+ HODL!'

The example comment is: `'Bury Burry!!!!!'`. Print out what you get. For reproducibility, use the same comment in the next step; consider setting a seed.

#### 4. Make Inference!

In [21]:
sentiment = sentiment_model(comment)

type(sentiment)

list

What is the type of the output `sentiment`?

```
list
```

In [23]:
print(f'The comment: {comment}')
print(f'Predicted Label is {sentiment[0]["label"]} and the score is {sentiment[0]["score"]:.3f}')

The comment: TSLA $2,500+ HODL!
Predicted Label is NEGATIVE and the score is 0.995


For the example comment, the output is:

    The comment: Bury Burry!!!!!
    Predicted Label is NEGATIVE and the score is 0.989

🖥️❓ Model Question:

1. What does the score represent? 

The score represents the accuracy of the the pre-trained model when measuring the training data with the test data.

### Task IV: Put All Together

Let's pull all the piece together, create a simple script that does 

- get the subreddit
- get comments from the top posts for given subreddit
- run sentiment analysis 

#### Complete the Script

Once you complete the code, running the following block writes the code into a new Python script and saves it as `top_tlsa_comment_sentiment.py` under the same directory with the notebook. 

In [24]:
%%writefile top_tlsa_comment_sentiment.py

import secrets_reddit
import random

from typing import Dict, List

from praw import Reddit
from praw.models.reddit.subreddit import Subreddit
from praw.models import MoreComments

from transformers import pipeline


def get_subreddit(display_name:str) -> Subreddit:
    """Get subreddit object from display name

    Args:
        display_name (str): [description]

    Returns:
        Subreddit: [description]
    """
    reddit = Reddit(
        client_id=secrets_reddit.REDDIT_API_CLIENT_ID,        
        client_secret=secrets_reddit.REDDIT_API_CLIENT_SECRET,
        user_agent=secrets_reddit.REDDIT_API_USER_AGENT
        )
    
    subreddit = reddit.subreddit(display_name)
    return subreddit

def get_comments(subreddit:Subreddit, limit:int=3) -> List[str]:
    """ Get comments from subreddit

    Args:
        subreddit (Subreddit): [description]
        limit (int, optional): [description]. Defaults to 3.

    Returns:
        List[str]: List of comments
    """
    top_comments = []
    for submission in subreddit.top(limit=limit):
        for top_level_comment in submission.comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            top_comments.append(top_level_comment.body)
    return top_comments

def run_sentiment_analysis(comment:str) -> Dict:
    """Run sentiment analysis on comment using default distilbert model
    
    Args:
        comment (str): [description]
        
    Returns:
        str: Sentiment analysis result
    """
    sentiment_model = pipeline("sentiment-analysis")
    sentiment = sentiment_model(comment)
    return sentiment[0]


if __name__ == '__main__':
    subreddit = get_subreddit('tsla')
    comments = get_comments(subreddit)
    comment = random.choice(comments)
    sentiment = run_sentiment_analysis(comment)
    
    print(f'The comment: {comment}')
    print(f'Predicted Label is {sentiment["label"]} and the score is {sentiment["score"]:.3f}')

Overwriting top_tlsa_comment_sentiment.py


Run the following block to see the output.

In [25]:
!python top_tlsa_comment_sentiment.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
The comment: People need to stop talking about a stock split. It’s not productive. With the ability to buy fractional shares from most brokerages, it isn’t needed. 

Realistically, the share price should be in the $800 range, and should have been coming out of earnings. The run to the $1200s was a gamma squeeze triggered by that Hertz announcement. No technical support for it. Where we are now is n

<details><summary> Expected output:</summary>

    No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
    The comment: When is DOGE flying
    Predicted Label is POSITIVE and the score is 0.689
</details>

💽❓ Data Question:

5. Is the subreddit active? About how many posts or threads per day? How could you find this information?

The subreddit is lightly active with at least 1 post a day. The average post per day in the past month have been 2.63.

In [101]:
import datetime

dates = {}

for submission in tsla_subreddit.new():
    date = datetime.date.fromtimestamp(submission.created_utc).strftime('%m-%d-%Y')
    if date in dates:
        dates[date] += 1
    else:
        dates[date] = 1
        
for date, posts in dates.items():
    print("Date:", date)
    print("Number of posts:", posts)
    print("----")
    
avg_post = sum(dates.values()) / len(dates.values())
print("Average post per day is", avg_post)

Date: 08-29-2022
Number of posts: 2
----
Date: 08-28-2022
Number of posts: 5
----
Date: 08-26-2022
Number of posts: 1
----
Date: 08-25-2022
Number of posts: 5
----
Date: 08-24-2022
Number of posts: 7
----
Date: 08-22-2022
Number of posts: 2
----
Date: 08-18-2022
Number of posts: 5
----
Date: 08-17-2022
Number of posts: 3
----
Date: 08-15-2022
Number of posts: 2
----
Date: 08-14-2022
Number of posts: 1
----
Date: 08-13-2022
Number of posts: 3
----
Date: 08-12-2022
Number of posts: 3
----
Date: 08-11-2022
Number of posts: 3
----
Date: 08-10-2022
Number of posts: 3
----
Date: 08-09-2022
Number of posts: 1
----
Date: 08-08-2022
Number of posts: 4
----
Date: 08-05-2022
Number of posts: 4
----
Date: 08-04-2022
Number of posts: 4
----
Date: 08-03-2022
Number of posts: 1
----
Date: 08-02-2022
Number of posts: 2
----
Date: 08-01-2022
Number of posts: 3
----
Date: 07-30-2022
Number of posts: 1
----
Date: 07-29-2022
Number of posts: 1
----
Date: 07-28-2022
Number of posts: 2
----
Date: 07-27-2022

💽❓ Data Question:

6. Does there seem to be a large distribution of posters or a smaller concentration of posters who are very active? What kind of impact might this have on the data?

Seems like there is a large distribution of posters and many of them have only posted onces. By having different posters, it can prevent bias that one user might impose on the subreddit. The large distributions allows proper representation 

In [100]:
authors = {}

for submission in tsla_subreddit.top(time_filter="all"):
    if submission.author in authors:
        authors[submission.author] += 1
    else:
        authors[submission.author] = 1
        
for author, posts in authors.items():
    print("Author:", author)
    print("Number of posts:", posts)
    print("----")

Author: TSLAinsider
Number of posts: 1
----
Author: MrBills07
Number of posts: 1
----
Author: bigjimz88
Number of posts: 3
----
Author: G_Train24
Number of posts: 1
----
Author: lxelite89
Number of posts: 1
----
Author: ThenickThenick
Number of posts: 1
----
Author: Kornbelly
Number of posts: 2
----
Author: Kaiserschmorrn
Number of posts: 2
----
Author: _feelsgoodman__
Number of posts: 1
----
Author: None
Number of posts: 4
----
Author: Coffeeandtrade
Number of posts: 1
----
Author: jrow68
Number of posts: 1
----
Author: jrventure1
Number of posts: 2
----
Author: BelmontMan
Number of posts: 1
----
Author: _Dannyio
Number of posts: 1
----
Author: DopeJeprox007
Number of posts: 1
----
Author: Newt_Gold
Number of posts: 1
----
Author: unleashthepotential
Number of posts: 1
----
Author: ExplanationGeneral31
Number of posts: 1
----
Author: Savings-Bee-2896
Number of posts: 2
----
Author: SantiagoCoffee
Number of posts: 2
----
Author: droneauto
Number of posts: 7
----
Author: Legal_Philosoph